In [1]:
%matplotlib notebook
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, IntSlider
import matplotlib.pyplot as plt
from PIL import Image
import time
import warnings
from time import sleep
#from numba import jit
warnings.filterwarnings('ignore')

In [2]:
lena = np.array(Image.open('resources/lena_gray.png'), dtype=np.uint8)

## Image Filtering - 2D Convolution

In this exercise you have to implement your own function to apply a 2D convolution to the grayscaled version of Lena. Write a function where you can apply a 2d convolution to an image with different kernels and return the result. You have to implement the following Kernels:

* 3 x 3 Box Filter
* 5 x 5 Box Filter
* 5 x 5 Gaussian Filter with sigma = 5
* 3 x 3 Sobel Filter in Y-Direction

To avoid pixel access outside the image borders, implement padding (with zeros) and mirroring as border handling methods.

![](resources/task_1.gif)


**Hints**
* Don't forget to normalize each kernel! 
* Clamp your values to $[0, 255]$ with `max(0, min(255, 0))`
* You can use the formular $h_{\sigma} = \mathrm{e}^{-\frac{x^2 + y^2}{2\sigma^2}}$ to calculate the Gaussian filter (CV_03_Spatial_Filtering slide 4-33) for a kernel $K_{m\times n}$. 
$K_y \in \mathbb{Z} \, | \, [\lfloor-\frac{K_{m}}{2}\rfloor, \lfloor\frac{K_{m}}{2}\rfloor], K_x \in \mathbb{Z} | [\lfloor-\frac{K_{n}}{2}\rfloor, \lfloor\frac{K_{n}}{2}\rfloor]$

* For border handling you can use [np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html)

### Solution

In [3]:
def build_2d_gauss(sigma, kernel_width):
    #kernel_width = sigma * 2 if (sigma * 2) & 1 != 0 else sigma * 2 + 1
    sigma = 2.0 * sigma*sigma    
    half = kernel_width // 2
    out = np.zeros((kernel_width, kernel_width))
    for y in range(-half, half + 1):
        for x in range(-half, half + 1):
            n = (x**2 + y**2)            
            out[y + half,x + half] = np.exp(-(n/sigma))
    return out / np.sum(out)

def build_2d_filter(filter_name):
    if filter_name == '3x3 Box':
        return np.ones((3,3)) / 9.0
    elif filter_name == '7x7 Box':
        return np.ones((7,7)) / 49
    elif filter_name == '5x5 Gaussian σ=5':
        return build_2d_gauss(3, 5)
    elif filter_name == '3x3 Sobel Y-Dir':
        sobel = np.array([[-1, -2, -1],[0,0,0],[1, 2, 1]])    
        return sobel / np.abs(sobel).sum()
    
#@jit
def convolve(image, filter_name, border='Padding', use_classic=False):
    kernel = build_2d_filter(filter_name)
    kx_half = kernel.shape[1] // 2
    ky_half = kernel.shape[0] // 2
    
    offset = 0
    if filter_name == '3x3 Sobel Y-Dir':
        offset = 128
    
    
    if border == 'Padding':
        pad_image = np.pad(image,((ky_half,ky_half),(kx_half,kx_half)))
    else:
        pad_image = np.pad(image,((ky_half,ky_half),(kx_half,kx_half)), mode='reflect')
        
    convolved_image = np.zeros(image.shape)
    
    if use_classic:    
        for y in range(0, image.shape[0]):
            for x in range(0, image.shape[1]):
                Y = 0.0
                for ky in range(-ky_half, ky_half + 1):
                    for kx in range(-kx_half, kx_half + 1):
                        Y += pad_image[y + ky + ky_half, x + kx + kx_half] * kernel[ky, kx]
                convolved_image[y,x] = max(0, min(255, Y + offset)) 
    else:
        for y in range(0, image.shape[0]):
            for x in range(0, image.shape[1]):                           
                mat = pad_image[y:y + kernel.shape[1], x: x + kernel.shape[0]]
                Y = np.multiply(mat, kernel).sum()
                Y = max(0, min(255, Y + offset)) 
                convolved_image[y,x] = Y
    return convolved_image

NameError: name 'kernel' is not defined

### Visualization

In [12]:
@interact(filter_name=['','3x3 Box','7x7 Box', '5x5 Gaussian σ=5', '3x3 Sobel Y-Dir'], border=['Padding', 'Mirroring'], use_classic=False)
def execute(filter_name='', border='', use_classic=False):
    if filter_name is "":
        return    
    
    start = time.time()
    convolved = convolve(lena.copy(), filter_name, border, use_classic)
    print(f"Execution time {(time.time() - start ) * 1000} ms")
    
    
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Source")
    plt.imshow(lena, cmap="gray", vmin=0, vmax=255, interpolation='bilinear')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title(filter_name)
    plt.imshow(convolved, vmin=0, vmax=255, cmap="gray",interpolation='bilinear')
    plt.axis('off')
    plt.show()    

interactive(children=(Dropdown(description='filter_name', options=('', '3x3 Box', '7x7 Box', '5x5 Gaussian σ=5…

##  Image Filtering - Median Filter

For this task we have to find a way to remove the bad looking <b>salt and pepper</b> noise from Lena. To solve this problem, you need to implement a median filter. Write a function to pass the desired kernel size, apply the median filter, return the filtered image, and display it.
<img src="resources/task_2.png" />


In [13]:
lena_noisy = np.array(Image.open('resources/lena_salt_and_pepper.png'), dtype=np.uint8)

### Solution

In [14]:
#@jit
def median(image, filter_size, border='reflect', use_classic=False):
    k_half = int(filter_size // 2)
    
    if border == 'Padding':
        pad_image = np.pad(image,((k_half,k_half),(k_half,k_half)))
    else:
        pad_image = np.pad(image,((k_half,k_half),(k_half,k_half)), mode='reflect')
        
    median_image = np.zeros(image.shape)    
    if use_classic:    
        for y in range(0, image.shape[0]):
            for x in range(0, image.shape[1]):
                Ys = np.zeros((filter_size,filter_size))
                for ky in range(-k_half, k_half + 1):
                    for kx in range(-k_half, k_half + 1):
                        Ys[kx, ky] = pad_image[y + ky, x + kx]                                
                median_image[y,x] = np.median(Ys)            
    else:
        for y in range(0, image.shape[0]):
            for x in range(0, image.shape[1]):                
                mat = pad_image[y:y + filter_size, x: x + filter_size].flatten()                
                median_image[y,x] = np.median(mat)
    return median_image

### Visualization

In [15]:
@interact(filter_size=IntSlider(min=3, max=11, value=3, step=2, continuous_update=False), use_classic=False)
def execute(filter_size=3.0, use_classic=False):
    start = time.time()
    filtered = median(lena_noisy, filter_size, 'reflect', use_classic)
    print(f"Execution Time {((time.time()-start) * 1000):.2f} ms")
    plt.figure(figsize=(9,5))
    plt.subplot(1,2,1)
    plt.title("Source")    
    plt.imshow(lena_noisy, cmap="gray", vmin=0, vmax=255, interpolation='bilinear')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title("Median %d x %d" %(filter_size, filter_size))
    plt.imshow(filtered, cmap="gray", vmin=0, vmax=255, interpolation='bilinear')
    plt.axis('off')
    plt.show() 

interactive(children=(IntSlider(value=3, continuous_update=False, description='filter_size', max=11, min=3, st…